# LLM for coding

In this mini tutorial we will demonstrate how LLM agents can be used effectively for coding.

We will use the previous notebook "loading_example" to see if with a few prompts LLM can reproduce our code and write tests for it 

In [1]:
# Data handling packages
import numpy as np  
import pandas as pd 
import pynwb  

# Plotting libraries
import matplotlib.pyplot as plt  

# Pandas display settings
pd.set_option('display.max_columns', None)  # Ensures all columns are shown when printing DataFrames

# Inline plotting for Jupyter Notebooks
%matplotlib inline  

In [ ]:
# pick a session_id and get session data
example_session_ids = ['664851_2023-11-15', '668755_2023-08-31', '674562_2023-10-03', '759434_2025-02-04', '713655_2024-08-09']

session_id = example_session_ids[2]
nwb_path = f'/root/capsule/data/{session_id}/{session_id}.nwb'

# access the session data with pynwb
session = pynwb.NWBHDF5IO(nwb_path).read()


### If you are using ChatGPT UI

Let’s begin with a simple example. Following the logic from the previous notebook, our goal is to extract session metadata and organize it into DataFrames.

Try the following prompt:

```"Given a session, load the metadata—specifically, the trials and units—into separate DataFrames."```

Before accepting the result, evaluate what went wrong. Did we give too little information? 

Try this prompt:

```"You are a neuroscientist helping me analyze data from a recent experiment. The data is organized into multiple sessions, each of which can be loaded using: `session = pynwb.NWBHDF5IO(nwb_path).read()`. From each session, I need to extract metadata—specifically, trials and units—and convert them into separate pandas DataFrames. Write a Python script to accomplish this."```

Notice how much more specific this prompt is—you can refine it even further. Try the following:

```"You are a neuroscientist helping me analyze data from a recent experiment. The data is organized into multiple sessions, each of which can be loaded with: `session = pynwb.NWBHDF5IO(nwb_path).read()`. I need to extract metadata from a session—specifically, trials and units—and convert them into separate pandas DataFrames. Assume the session is already loaded. Write a concise Python script that performs this task. Do not save the DataFrames to CSV. Return only the script, with no additional explanation."```

Discuss: Can you think of a shorter prompt to produce the same or similar result? 


### If you are using Copilot

Let’s begin with a simple example. Following the logic from the previous notebook, our goal is to extract session metadata and organize it into DataFrames.

Try the following prompt:

```"Given a session, load the metadata—specifically, the trials and units—into separate DataFrames."```

Discuss: Why does this prompt work well with GitHub Copilot but not as effectively in the ChatGPT UI?

With Copilot, short and specific code-related instructions often lead to immediate and relevant completions, since it's optimized for in-context code generation. Let's test this difference by attempting a practical example: generating a function and then writing a test for it.


 
We’ll start by providing only the function description and see if Copilot can infer and generate the correct implementation. Try the following prompt:

```"Generate a function called `make_psth` with the following behavior: The function should compute a Peri-Stimulus Time Histogram (PSTH).```

```Parameters:```
   - spike_times: array-like, timestamps of all spikes (in seconds)
   - stim_times: array-like, timestamps of stimulus onsets (in seconds)
   - pre_window: float, time before stimulus to include in the PSTH (seconds)
   - post_window: float, time after stimulus to include in the PSTH (seconds)
   - bin_size: float, width of each time bin (seconds)

```Returns:```
   - firing_rates: 2D NumPy array of firing rates (shape: trials × bins)
   - bin_centers: 1D NumPy array of bin center times (relative to stimulus onset)"

How can we verify that this function works as intended?
One way is to write a test to ensure that make_psth executes properly and returns meaningful output. Try the following prompt:

```"Generate a test to ensure the function `make_psth` runs correctly and returns meaningful results."```

How do you know the test result is actually meaningful? Compare the result with a person sitting next to you. You should _always_ go back and verify the output yourself—never assume it's correct just because it runs.

Alternatively, for smaller code blocks or quick autocompletions, you can use Copilot to assist with implementation.

Exercise. Rewrite the function above using autocomplete. Make sure to check every line the Copilot generates for you. 